In [ ]:
import pyswmm
from pyswmm import Simulation,Subcatchments,Nodes,SystemStats,LidControls,LidGroups
import numpy as np
import pandas as pd
import math
import queue
import threading
import subprocess
import time
import random
import ast


In [ ]:
# Parameters

area2_range=[0.02,0.2]

# Surface layer thickness
surface_thickness=[100,600]

# soil layer thickness
soil_thickness=[300,600]

#Storage layer thickness
storage_thickness=[800,2000]

#Drain offset
#drain_offset=[0,200,400,600,800,1000]
drain_offset=[0,1000]

#Soil layer conductivity
seepage_rate=[0.5,120]

In [ ]:
# The number of parameter combination (simulations) to be done
num_iterations=1000

In [ ]:
# Create a queue and add all the simulations to it
simulations_queue = queue.Queue()
for _ in range(num_iterations):
        params = (
            round(random.uniform(min(area2_range), max(area2_range)), 2),
            round(random.uniform(min(surface_thickness), max(surface_thickness)), 2),
            round(random.uniform(min(soil_thickness), max(soil_thickness)), 2),
            round(random.uniform(min(storage_thickness), max(storage_thickness)), 2),
            random.choice(drain_offset),
            round(random.uniform(min(seepage_rate), max(seepage_rate)), 2)
        )
        simulations_queue.put(params)
#simulations_queue.get()


In [ ]:
# Create empty lists to save the results in them
Results_area2=[]
Results_Bio_surface_thickness=[]
Results_Bio_soil_thickness=[]
Results_Bio_storage_thickness=[]
Results_Bio_drain_offset=[]
Results_Bio_seepage_rate=[]
Results_S1_stat=[]
Results_S2_stat=[]
Results_system_stat=[]

In [ ]:
# Function to process the simulations from the queue
def process_simulation_queue():
    while not simulations_queue.empty():
        sim_params = simulations_queue.get()
        result = subprocess.run(["python", "pyswmm_wrapper.py", "SWMM.inp",
                                 str(sim_params[0]),  # A2
                                 str(sim_params[1]),  # Surface_thickness_value
                                 str(sim_params[2]),  # Soil_thickness_value
                                 str(sim_params[3]),  # Storage_thickness_value
                                 str(sim_params[4]),  # Drain_offset_value
                                 str(sim_params[5])   # Seepage_rate_value
                                ], capture_output=True, text=True)
        
        #print(result)
        #print("---------------------------------------------------------------------------------------------------")
        
                                 
        if result.returncode == 0:
            #print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            output_lines = result.stdout.splitlines()
            #print(output_lines)
            Results_area2.append(float(output_lines[0]))           
            Results_Bio_surface_thickness.append(float(output_lines[1]))
            Results_Bio_soil_thickness.append(float(output_lines[2]))
            Results_Bio_storage_thickness.append(float(output_lines[3]))
            Results_Bio_drain_offset.append(float(output_lines[4]))
            Results_Bio_seepage_rate.append(float(output_lines[5]))
            Results_S1_stat.append(output_lines[6])
            Results_S2_stat.append(output_lines[7])
            Results_system_stat.append(output_lines[8])

        simulations_queue.task_done()
        

In [ ]:
# Create and start the threads
num_threads = 20  # Adjust the number of threads as per your system's capabilities
threads = []
start_time=time.time()
for _ in range(num_threads):
    thread = threading.Thread(target=process_simulation_queue)
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()
    
print("Finish")
end_time=time.time()

In [ ]:
elapsed_time = (end_time - start_time)/3600

print(f"Function took {elapsed_time:.6f} hrs to execute.")

In [ ]:
# You need to change the returned values from string to float
Results_area2_float=[float(i) for i in Results_area2]


In [ ]:
# You need to change the returned values from string to dictionaries 
Results_system_stat_dict=[]
Results_S1_stat_dict=[]
Results_S2_stat_dict=[]

for index,item in enumerate(Results_system_stat):
    Results_system_stat_dict.append(ast.literal_eval(Results_system_stat[index]))
for index,item in enumerate(Results_S1_stat):
    Results_S1_stat_dict.append(ast.literal_eval(Results_S1_stat[index]))
for index,item in enumerate(Results_S2_stat):
    Results_S2_stat_dict.append(ast.literal_eval(Results_S2_stat[index]))

In [ ]:
# Then we can save all the returned values in a dataframe for a further analysis
Results_df=pd.DataFrame()
Results_df['Area_S2']=Results_area2_float
Results_df['Bio_Berm_Height']=Results_Bio_surface_thickness
Results_df['Bio_soil_thickness']=Results_Bio_soil_thickness
Results_df['Bio_storage_thickness']=Results_Bio_storage_thickness
Results_df['Bio_drain_thickness']=Results_Bio_drain_offset
Results_df['Bio_seepage_rate']=Results_Bio_seepage_rate
Results_df['S1_statistics']=Results_S1_stat_dict
Results_df['S2_statistics']=Results_S2_stat_dict
Results_df['System_statistics']=Results_system_stat_dict
Results_df